In [1]:
import numpy as np
import json
import matplotlib.pyplot as plt
from cieluv import *
from RootPolyRegression import *
%matplotlib inline

In [2]:
patch_spectras = ['Dark Skin.reflect', 'Light Skin.reflect', 'Blue Sky.reflect',
                 'Foilage.reflect', 'Blue Flower.reflect', 'Bluish Green.reflect',
                 'Orange.reflect', 'Purplish Blue.reflect', 'Moderate Red.reflect',
                 'Purple.reflect', 'Yellow Green.reflect', 'Orange Yellow.reflect',
                 'Blue.reflect', 'Green.reflect', 'Red.reflect',
                 'Yellow.reflect', 'Magenta.reflect', 'Cyan.reflect',
                 'White.reflect', 'Neutral-8.reflect', 'Neutral-6.5.reflect',
                 'Neutral-5.reflect', 'Neutral-3.5.reflect', 'Black.reflect']

In [3]:
def LOO(Q, R, whitepoint):
    err = 0
    
    for i in range(R.shape[1]):
        
        # i-ый объект на тесте, остальные на трейне
        Q_train = np.concatenate((Q[:, :i], Q[:, (i+1):]), axis=1)
        R_train = np.concatenate((R[:, :i], R[:, (i+1):]), axis=1)
        Q_test = Q[:, i]
        R_test = R[:, i]
        
        # fitting
        A = np.dot(np.dot(Q_train, R_train.T), np.linalg.inv(np.dot(R_train, R_train.T)))
        
        # score on test object
        err += CIELUV_DeltaE_from_XYZ(np.dot(A, R_test), Q_test, whitepoint)
        
    return err / 24

In [4]:
x = np.linspace(380, 780, 101)

# D65 source
src = np.loadtxt("./d65.illum")
x_src = np.linspace(300, 830, 531)
src = np.interp(x, x_src, src)
D65 = np.diag(src)

# XYZ spectral sensitivities
f = open('camera.kinect1.json', 'r')
xyz_dict = json.load(f)
XYZ_R = np.array(xyz_dict['sensitivities']['red'])[:, 1]
XYZ_G = np.array(xyz_dict['sensitivities']['green'])[:, 1]
XYZ_B = np.array(xyz_dict['sensitivities']['blue'])[:, 1]
x_XYZ = np.linspace(365, 830, 94) 
sens_XYZ_R = np.interp(x, x_XYZ, XYZ_R)
sens_XYZ_G = np.interp(x, x_XYZ, XYZ_G)
sens_XYZ_B = np.interp(x, x_XYZ, XYZ_B)

# RGB camera sensitivities
cam_R, cam_G, cam_B = np.loadtxt("cam_R.spect"), np.loadtxt("cam_G.spect"), np.loadtxt("cam_B.spect")
x_RGB = np.linspace(380, 784, 101)
sens_RGB_R = np.interp(x, x_RGB, cam_R)
sens_RGB_G = np.interp(x, x_RGB, cam_G)
sens_RGB_B = np.interp(x, x_RGB, cam_B)


In [5]:
# Get white point

X_WHITE = np.dot(sens_XYZ_R.T, src)
Y_WHITE = np.dot(sens_XYZ_G.T, src)
Z_WHITE = np.dot(sens_XYZ_B.T, src)
print("White point: ", X_WHITE, Y_WHITE, Z_WHITE)
WHITE_POINT_XYZ_D65 = [X_WHITE, Y_WHITE, Z_WHITE]
WHITE_POINT_XYZ_D65 /= Y_WHITE
WHITE_POINT_XYZ_D65

White point:  2754.907432227824 2905.2600332985335 3118.0681991097817


array([0.94824814, 1.        , 1.07324927])

In [15]:
# Calculate trisitumules
colorsXYZ = []
for patch in patch_spectras:
    refl = np.loadtxt("./patches/" + patch)
    X = np.dot(np.dot(sens_XYZ_R.T, D65), refl)
    Y = np.dot(np.dot(sens_XYZ_G.T, D65), refl)
    Z = np.dot(np.dot(sens_XYZ_B.T, D65), refl)
    c = np.array([X, Y, Z]) / Y_WHITE
    colorsXYZ.append(c)
colorsXYZ = np.array(colorsXYZ)

colorsRGB = []
for patch in patch_spectras:
    refl = np.loadtxt("./patches/" + patch)
    R = np.dot(np.dot(sens_RGB_R.T, D65), refl)
    G = np.dot(np.dot(sens_RGB_G.T, D65), refl)
    B = np.dot(np.dot(sens_RGB_B.T, D65), refl)
    c = np.array([R, G, B]) / Y_WHITE
    colorsRGB.append(c)
colorsRGB = np.array(colorsRGB)

Root-polinomial

In [16]:
# R -- RGB matrix FxN, F -- number of features, N -- objects
# Q -- XYZ matrix 3xN


for power in range(1, 5):
    R = get_rp_features(colorsRGB.T, power) # returns root-poly features
    Q = colorsXYZ.T
    print("power: ", power, " score: ", LOO(Q, R, WHITE_POINT_XYZ_D65))

power:  1  score:  2.9219445148647085
power:  2  score:  1.75559856750236
power:  3  score:  3.0991345550664895
power:  4  score:  3266.625979366528


Polynomial

In [17]:
from sklearn.preprocessing import PolynomialFeatures

for power in range(1, 5):
    poly = PolynomialFeatures(power)
    R = poly.fit_transform(colorsRGB).T
    Q = colorsXYZ.T
    print("power: ", power, " score: ", LOO(Q, R, WHITE_POINT_XYZ_D65))

power:  1  score:  3.1271458060963275
power:  2  score:  2.9908176074392814
power:  3  score:  11.507637292746933
power:  4  score:  19610.734645866378


In [18]:
for power in range(1, 5):
    poly = PolynomialFeatures(power, include_bias=False)
    R = poly.fit_transform(colorsRGB).T
    Q = colorsXYZ.T
    
    print("power: ", power, " score: ", LOO(Q, R, WHITE_POINT_XYZ_D65))

power:  1  score:  2.9219445148647054
power:  2  score:  2.8322044377666256
power:  3  score:  10.84702063677562
power:  4  score:  847.756684140318
